## Librerias a usar

In [12]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import random

## Procesado de datos

Estos datos tienen que ser en formato de lista

In [13]:
productos = ['pan', 'leche', 'huevos', 'queso', 'yogurt', 'manzanas', 'plátanos', 'naranjas', 'pollo', 'carne', 'pescado', 'arroz', 'pasta', 'cereal', 'café', 'té', 'azúcar', 'sal', 'aceite',  'galletas', 'chocolate', 'papas fritas', 'refresco', 'agua', 'cerveza']

num_transacciones = 1000
transacciones = []

## Funcion para generar los candidatos

Generamos combinaciones de items de tamaño k

In [14]:
#Creo transacciones aleatorias
for i in range(num_transacciones):
    num_productos = random.randint(1, 8) # N° productos entre 1 y 8

    transaccion = random.sample(productos, num_productos) #con esto selecciono bajo aleatoridad

    transacciones.append(transaccion)

#Aqui empiezo con las reglas dde asociacion y simular patrones de compra
#Regla 1: Si hay pan, agrega leche con 80% de prob. (esto dentro de las primeras 200 transacciones)
for i in range(200):
    if 'pan' in transacciones[i] and random.random() < 0.8:
        if 'leche' not in transacciones[i]:
            transacciones[i].append('leche')

#Regla 2: Si hay pasta, agrega salsa de tomate con 70% prob. (este rango se aumenta de 200 a 400)
for i in range(200, 400):
    if 'pasta' in transacciones[i] and random.random() < 0.7:
        if 'salsa de tomate' not in transacciones[i]:
            transacciones[i].append('salsa de tomate')

#Regla 3: Si hay cervesa, agrega papas fritas con 75% prob. (igual se aumenta de 400 a 600)
for i in range(400, 600):
    if 'cerveza' in transacciones[i] and random.random() < 0.75:
        if 'papas fritas' not in transacciones[i]:
            transacciones[i].append('papas fritas')

#Transformamos a formato one hot enconding
te = TransactionEncoder() #codifica las transacciones
te_ary = te.fit(transacciones).transform(transacciones) #cogemos los elementos unicos de todas las transacciones y convierte cada transaccion en un array booleano donde True = presente, False = no presente
df = pd.DataFrame(te_ary, columns=te.columns_) #crea el dataframe

In [ ]:
df
#Este metodo de codificacion es muy caro a nivel de costo computacional 

,aceite,agua,arroz,azúcar,café,carne,cereal,cerveza,chocolate,galletas,...,pasta,pescado,plátanos,pollo,queso,refresco,sal,salsa de tomate,té,yogurt
0,True,False,True,False,False,False,False,False,False,False,...,True,True,False,False,False,False,True,False,False,False
1,False,False,True,False,False,False,True,True,False,True,...,True,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,False,True,False,False,False,False,False,True,False,True,...,False,False,False,False,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
996,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,True
997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
df.to_csv('market_basket_dataset.csv', index=False)

print("Primeras 5 transacciones:")
for i in range(5):
    print(f"Transacción {i+1}: {transacciones[i]}")

print("\nDimensiones del dataset:", df.shape)

frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True) #con el min, los itemsets deben aparecer en al menos 5% y el column=true usa los nombres de los prod. en vez indices numericos.

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3) #asso rules, extrae las reglas a partir de los itemsets frecuentes, metric = filtra por confianza minima, threhold son las reglas co confianza mayor oigual a 30%

print("\nReglas de asociación más fuertes:")
print(rules.sort_values('confidence', ascending=False).head(10))

Primeras 5 transacciones:
Transacción 1: ['leche', 'arroz', 'pescado', 'aceite', 'pasta', 'sal']
Transacción 2: ['arroz', 'cerveza', 'pasta', 'galletas', 'cereal']
Transacción 3: ['refresco']
Transacción 4: ['galletas', 'huevos', 'agua', 'queso', 'cerveza']
Transacción 5: ['queso', 'pescado', 'leche']

Dimensiones del dataset: (1000, 26)

Reglas de asociación más fuertes:
      antecedents     consequents  antecedent support  consequent support  \
0       (cerveza)  (papas fritas)               0.178               0.184   
1  (papas fritas)       (cerveza)               0.184               0.178   
2           (pan)         (leche)               0.191               0.204   

   support  confidence      lift  representativity  leverage  conviction  \
0    0.060    0.337079  1.831949               1.0  0.027248    1.230915   
1    0.060    0.326087  1.831949               1.0  0.027248    1.219742   
2    0.061    0.319372  1.565548               1.0  0.022036    1.169508   

   zhangs_m